# Algoritmo Genético - resolução do problema da mochila
Este trabalho foi desenvolvido como requisito parcial da disciplina de Inteligência Artificial, com o objetivo de aplicar conceitos de otimização utilizando Algoritmos Genéticos (AGs).

O Problema da Mochila (Knapsack Problem) é um problema clássico de otimização combinatória no qual se deseja selecionar, dentre um conjunto de itens disponíveis, aqueles que proporcionam o maior valor total, respeitando restrições de capacidade.

#### Trabalho feito por: Davi Henrique, Kaike Alvarenga e Layla Melo

In [ ]:
# comando para instalar as dependencias necessarias:
# pip install numpy pandas matplotlib seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Configuração de estilo para os gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Produtos de investimento disponíveis
produtos = [
    {"nome": "Ação Tech A", "retorno": 15.5, "risco": 8.2, "custo": 5000},
    {"nome": "Ação Tech B", "retorno": 12.3, "risco": 6.5, "custo": 4500},
    {"nome": "Fundo Imobiliário", "retorno": 9.8, "risco": 4.3, "custo": 3000},
    {"nome": "Tesouro Direto", "retorno": 6.5, "risco": 1.5, "custo": 1000},
    {"nome": "CDB Banco", "retorno": 7.2, "risco": 2.0, "custo": 2000},
    {"nome": "Ação Varejo", "retorno": 11.5, "risco": 7.0, "custo": 3500},
    {"nome": "ETF Internacional", "retorno": 13.2, "risco": 6.8, "custo": 4000},
    {"nome": "Fundo Multimercado", "retorno": 10.5, "risco": 5.5, "custo": 3200},
    {"nome": "Debênture", "retorno": 8.5, "risco": 3.8, "custo": 2500},
    {"nome": "Cripto Index", "retorno": 18.5, "risco": 9.5, "custo": 6000},
    {"nome": "Ação Energia", "retorno": 10.8, "risco": 4.9, "custo": 3800},
    {"nome": "Ação Banco A", "retorno": 9.5, "risco": 4.1, "custo": 4100},
    {"nome": "Ação Banco B", "retorno": 8.8, "risco": 3.9, "custo": 3900},
    {"nome": "ETF S&P500", "retorno": 12.8, "risco": 5.7, "custo": 4200},
    {"nome": "ETF Brasil 100", "retorno": 10.2, "risco": 6.0, "custo": 3000},
    {"nome": "Fundo Cambial", "retorno": 7.5, "risco": 3.0, "custo": 2000},
    {"nome": "Fundo Previdência", "retorno": 6.0, "risco": 2.2, "custo": 1800},
    {"nome": "LCI Banco", "retorno": 7.0, "risco": 1.8, "custo": 1500},
    {"nome": "LCA Banco", "retorno": 7.1, "risco": 1.7, "custo": 1600},
    {"nome": "Ação Mineração", "retorno": 14.2, "risco": 7.8, "custo": 5200},
    {"nome": "Ação Siderurgia", "retorno": 13.0, "risco": 7.1, "custo": 4800},
    {"nome": "Ação Petróleo", "retorno": 16.5, "risco": 8.9, "custo": 5600},
    {"nome": "Ação Agronegócio", "retorno": 12.1, "risco": 5.9, "custo": 3400},
    {"nome": "Criptomoeda A", "retorno": 22.0, "risco": 12.0, "custo": 3000},
    {"nome": "Criptomoeda B", "retorno": 19.2, "risco": 10.1, "custo": 2800},
    {"nome": "Criptomoeda C", "retorno": 25.0, "risco": 13.5, "custo": 3500},
    {"nome": "Ouro", "retorno": 8.0, "risco": 3.4, "custo": 4500},
    {"nome": "Prata", "retorno": 7.5, "risco": 3.8, "custo": 3000},
    {"nome": "Diamante Sintético", "retorno": 11.0, "risco": 5.2, "custo": 7000},
    {"nome": "Fundo ESG", "retorno": 9.7, "risco": 4.5, "custo": 3200},
    {"nome": "Fundo Tecnologia Global", "retorno": 14.7, "risco": 7.3, "custo": 5000},
    {"nome": "Fundo Small Caps", "retorno": 16.2, "risco": 9.1, "custo": 4300},
    {"nome": "Startup Pré-IPO A", "retorno": 30.0, "risco": 15.0, "custo": 8000},
    {"nome": "Startup Pré-IPO B", "retorno": 27.5, "risco": 14.2, "custo": 7500},
    {"nome": "CDB Bancão", "retorno": 6.8, "risco": 1.2, "custo": 2500},
    {"nome": "CDB Médio Prazo", "retorno": 7.3, "risco": 2.3, "custo": 2200},
    {"nome": "Tesouro IPCA+", "retorno": 5.9, "risco": 1.0, "custo": 1500},
    {"nome": "Tesouro Selic", "retorno": 5.5, "risco": 1.0, "custo": 1000}
]

# Restrições do problema
ORCAMENTO_MAX = 15000  # Orçamento máximo em reais
MAX_PRODUTOS = 6       # Número máximo de produtos
RISCO_MAX = 35         # Risco máximo aceitável
# Parametro do Algoritmo
TAM_POPULACAO = 50      # Tamanho da população
NUM_GERACOES = 100      # Número de gerações
TAXA_CROSSOVER = 0.8    # Taxa de crossover (80%)
TAXA_MUTACAO = 0.05     # Taxa de mutação (5%)

In [ ]:
"""
    Calcula o fitness de um cromossomo.
    Fitness = Retorno Total - Penalidades
    Penalidades:
    - Excesso de orçamento × 0.5
    - Excesso de risco × 2.0
    - Excesso de produtos × 10
    """
def calcular_fitness(cromossomo):
    retorno_total = 0
    risco_total = 0
    custo_total = 0
    num_produtos = 0
    # Calcula totais
    for i, gene in enumerate(cromossomo):
        if gene == 1:
            retorno_total += produtos[i]["retorno"]
            risco_total += produtos[i]["risco"]
            custo_total += produtos[i]["custo"]
            num_produtos += 1
    # Calcula penalidades
    penalidade = 0
    if custo_total > ORCAMENTO_MAX:
        penalidade += (custo_total - ORCAMENTO_MAX) * 0.5
    if risco_total > RISCO_MAX:
        penalidade += (risco_total - RISCO_MAX) * 2.0
    if num_produtos > MAX_PRODUTOS:
        penalidade += (num_produtos - MAX_PRODUTOS) * 10
    # Fitness final
    fitness = retorno_total - penalidade
    return max(0, fitness)  # Garante que fitness não seja negativo


In [ ]:
"""
    Cria população inicial de forma aleatória.
    Probabilidade de 30% de cada gene ser 1 (selecionado).
"""
def criar_populacao_inicial(tamanho): 
    populacao = []
    for _ in range(tamanho):
        cromossomo = np.random.choice([0, 1], size=len(produtos), p=[0.7, 0.3])
        populacao.append(cromossomo)
    return populacao

In [ ]:
"""
    Seleção por Torneio.
    Escolhe k indivíduos aleatórios e retorna o melhor.
"""
def selecao_torneio(populacao, fitness, k=3):
    indices = np.random.choice(len(populacao), k, replace=False)
    fitness_torneio = [fitness[i] for i in indices]
    melhor_idx = indices[np.argmax(fitness_torneio)]
    return populacao[melhor_idx].copy()
"""
    Crossover de 1 ponto.
    Com probabilidade 'taxa', gera dois filhos a partir de um ponto de corte.
"""
def crossover_um_ponto(pai1, pai2, taxa):
    if np.random.random() > taxa:
        return pai1.copy(), pai2.copy()
    ponto = np.random.randint(1, len(pai1))
    filho1 = np.concatenate([pai1[:ponto], pai2[ponto:]])
    filho2 = np.concatenate([pai2[:ponto], pai1[ponto:]])
    return filho1, filho2
"""
    Mutação por inversão de bit.
    Cada gene tem probabilidade 'taxa' de ser invertido (0->1 ou 1->0).
"""
def mutacao_bit_flip(cromossomo, taxa):
    cromossomo_mutado = cromossomo.copy()
    for i in range(len(cromossomo_mutado)):
        if np.random.random() < taxa:
            cromossomo_mutado[i] = 1 - cromossomo_mutado[i]
    return cromossomo_mutado

In [ ]:
"""
    Executa o Algoritmo Genético completo.
    Retorna a população final e o histórico de evolução.
 """
def executar_ag(tam_pop=TAM_POPULACAO, num_ger=NUM_GERACOES, taxa_cross=TAXA_CROSSOVER, taxa_mut=TAXA_MUTACAO, verbose=True):
    # Cria população inicial
    populacao = criar_populacao_inicial(tam_pop)
    # Histórico para análise
    historico = {
        'geracao': [],
        'melhor_fitness': [],
        'fitness_medio': [],
        'fitness_pior': []
    }
    # Loop principal
    for geracao in range(num_ger):
        # Calcula fitness de todos os indivíduos
        fitness = [calcular_fitness(cromossomo) for cromossomo in populacao]
        # Guarda estatísticas
        historico['geracao'].append(geracao)
        historico['melhor_fitness'].append(max(fitness))
        historico['fitness_medio'].append(np.mean(fitness))
        historico['fitness_pior'].append(min(fitness))
        # Mostra progresso
        if verbose and (geracao % 20 == 0 or geracao == num_ger - 1):
            print(f"Geração {geracao:3d} | Melhor: {max(fitness):6.2f} | Médio: {np.mean(fitness):6.2f}")
        # Nova população (com elitismo)
        melhor_idx = np.argmax(fitness)
        nova_populacao = [populacao[melhor_idx].copy()]  # Elitismo
        # Gera novos indivíduos
        while len(nova_populacao) < tam_pop:
            # Seleção
            pai1 = selecao_torneio(populacao, fitness)
            pai2 = selecao_torneio(populacao, fitness)
            # Crossover
            filho1, filho2 = crossover_um_ponto(pai1, pai2, taxa_cross)
            # Mutação
            filho1 = mutacao_bit_flip(filho1, taxa_mut)
            filho2 = mutacao_bit_flip(filho2, taxa_mut)
            # Adiciona à nova população
            nova_populacao.append(filho1)
            if len(nova_populacao) < tam_pop:
                nova_populacao.append(filho2)
        populacao = nova_populacao
    return populacao, historico